First steps towards a sample of spec/photometric Ias. 

Final goal will be to have saltfits (based on both bts, ampel-spec and ampel-photo z), host dist/mass (LS?) and BTS type for a large sample.

Steps:
* Create a filter query and process candidates. (this notebook)
* Run a bunch of T2s (next).


Q1: How long does it take to process one week of alerts? (and how many do that end up being)
Roughly started at 14.40 Let it run to 15.30. At that time roughly a third of ~15000 chunks were processed. Saved O(200) transients. Seems very inefficient, since the same transients can be processed many times. The token request goes faster, but it seems like the field based works "fast enough".

Q2: How long does it take to process one field? Started roughly 15.30. Took ~15 min for the token to yield alerts. Ended up with ~150 chunks. This took ~20 min to process and led to 80 saved candidates. O(800) T2 tickets created. This processing started ~16.10. 

Conclusion: we will go through each field (as many as possible) and try to parse data and run t2s in parallel. 

As a check, rerun the same thing but using a topic stream based on the BTS names. What fraction would we see here?

In [ ]:
# These are fields which extend beyond 14 deg from the galactic plane:
fields = [  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  28,  29,  30,  31,
        32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       137, 138, 139, 140, 141, 142, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 190, 191,
       192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204,
       205, 206, 207, 208, 209, 210, 211, 212, 218, 219, 220, 221, 222,
       223, 224, 225, 226, 227, 228, 229, 230, 236, 237, 238, 239, 240,
       241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253,
       254, 255, 256, 257, 258, 264, 265, 266, 267, 268, 269, 270, 271,
       272, 273, 274, 275, 276, 277, 278, 279, 285, 286, 287, 288, 289,
       290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302,
       303, 304, 305, 306, 307, 313, 314, 315, 316, 317, 318, 319, 320,
       321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 336, 337, 338,
       339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351,
       352, 353, 354, 355, 356, 357, 363, 364, 365, 366, 367, 368, 369,
       370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 387,
       388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400,
       401, 402, 403, 404, 405, 406, 407, 408, 413, 414, 415, 416, 417,
       418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430,
       431, 432, 433, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448,
       449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 464, 465, 466,
       467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479,
       480, 481, 482, 483, 484, 485, 491, 492, 493, 494, 495, 496, 497,
       498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510,
       515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527,
       528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 543, 544, 545,
       546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558,
       559, 560, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576,
       577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 594,
       595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607,
       608, 609, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625,
       626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 643,
       644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 662,
       663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675,
       676, 677, 678, 679, 680, 681, 682, 683, 684, 691, 692, 693, 694,
       695, 696, 697, 698, 699, 706, 707, 708, 709, 710, 711, 712, 713,
       714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726,
       727, 734, 735, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755,
       756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 781, 782,
       783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795,
       796, 797, 798, 799, 800, 812, 813, 814, 815, 816, 817, 818, 819,
       820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 837, 838, 839,
       840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 854,
       855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867,
       868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879]

In [ ]:
fieldindex = 449   # We will process the fields in order
field_length = 10
do_fields = fields[fieldindex:fieldindex+field_length]
dbname = 'allIa'
channel = 'allIav1'

In [ ]:
print('We will process field %s'%(do_fields))

Stopping here _after_ field 598. To keep as "training" sample (and allow T2s to catch up). 

In [ ]:
import sys, os, time
import requests
from requests.exceptions import HTTPError
from requests.exceptions import ConnectionError

In [ ]:
from ampel.secret.AmpelVault import AmpelVault
from ampel.secret.DictSecretProvider import DictSecretProvider
from ampel.dev.DevAmpelContext import DevAmpelContext
from ampel.ztf.ingest.ZiCompilerOptions import ZiCompilerOptions

In [ ]:
# Create a secret vault
secrets = { 'slack/ztf_general/jno': os.environ["SNIaSLACK"],
                              'mongo/logger':{},
                              'mongo/writer':{},
                              'extcats/reader':{},
            'ztf/archive/auth' : {'user' : '', 'password' : ''},
            'datalab/pwd':os.environ["DATALABPWD"],
            "desycloud" : {},
            "ztf/archive/token": os.environ["ARCHIVE_TOKEN"],
          }
vault = AmpelVault([DictSecretProvider(secrets)])

In [ ]:
header = {"Authorization": "bearer "+vault.get_named_secret("ztf/archive/token").value}
endpoint = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3/streams/from_query?programid=1'

In [ ]:
query = {
  "jd": {
    "$gt": 0.,
    "$lt": 24588580.,
  },
  "candidate": {
    "rb": {
      "$gt": 0.3
    },
    "magpsf": {
      "$gt": 17
    },
    "ndethist": {
      "$gt": 8,
    },
    "distpsnr1": {
      "$lte": 12             # Otherwise no redshift anyway? Except for nearby?
    },
    "isdiffpos": {"$in": ["t", "1"]},
#    "field": {"$eq":fields[fieldindex]},
    "field": {"$in":do_fields},
  }
}

In [ ]:
query

In [ ]:
response = requests.post(endpoint, headers=header, json=query )
if not response.ok:
    print( 'Query creation failed' )
    print(response)

In [ ]:
loader_config = {'archive':"https://ampel.zeuthen.desy.de/api/ztf/archive/v3", "stream":response.json()['resume_token']}

In [ ]:
loader_config 

In [ ]:
loader_config = {'archive': 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3',
 'stream': 'V8RLaFsBb2i2wLRlr6quLkc8lxRLdFS4jfdVws0Ufws'}

In [ ]:
AMPEL_CONF = '/home/jnordin/github/ampel82/ampel_conf_630434.yaml'

In [ ]:
# Select date of query with format e.g. '20210610'
ITER_MAX = 1000000

In [ ]:
ctx = DevAmpelContext.load(
    config = AMPEL_CONF,
    db_prefix = dbname,
    purge_db = False,
    vault = vault
)
ctx.add_channel(
    name=channel,
    access=['ZTF', 'ZTF_PUB', 'ZTF_PRIV']
)

In [ ]:
# For accessing the datalab portal to LS photo-z. Replace the username with what matches with the password 
# entered above
dl_config = {
    'datalab_user' : "wombaugh",
    'datalab_pwd' : vault.get_named_secret("datalab/pwd").value,
}

In [ ]:
# These are configuration parameters for the initial alert filter step.
# Note that this filter configuration assumes that a first filter based on nbr of points were made in the alert query
# The T0 stage will not be exposed to the full candidate history. 
filter_config = {
    'min_ndet': 1,
    'min_tspan': 15,
    'max_tspan': 200,
    'min_archive_tspan': 15,
    'max_archive_tspan': 200,    
    'min_rb': 0.3,
    'max_fwhm': 5.5,
    'min_gal_lat': 14,
    'ps1_sgveto_rad': 1,
    'ps1_sgveto_th': 0.8,
    'ps1_confusion_rad': 3,
    'ps1_confusion_sg_tol': 0.1
}

In [ ]:
# Will use NED for spectroscopic redshifts.
cat_conf = {
    'catalogs' : {
        'SDSS_spec' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['z', 'bptclass', 'subclass'],
            'all': False,
        },
        'NEDz' : {
            'use' : 'catsHTM',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['ObjType', 'Velocity', 'z'],
        },
        'GLADEv23' : {
            'use' : 'extcats',
            'rs_arcsec' : 10,
            'keys_to_append' : ['z', 'dist', 'dist_err', 'flag1', 'flag2', 'flag3'],
        },
        'LSPhotoZZou' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['photoz','ra','dec','e_photoz','specz','_6','logMassBest','logMassInf','logMassSup'],
            'pre_filter' : None,
            'post_filter' : None,
            'all': False,
        },
        'wiseScosPhotoz' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['zPhoto_Corr','ra','dec','wiseID','w1mCorr','w2mCorr'],
            'pre_filter' : None,
            'post_filter' : None,
        },
        'twoMPZ' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['zPhoto','ra','dec','zSpec'],
            'pre_filter' : None,
            'post_filter' : None,
        },

    }
}

In [ ]:
# New, new, also with all categories?
ampelz_conf = {
    "max_redshift_category" : 7,
    "t2_dependency": [
        {
            "unit": "T2CatalogMatch",
            "config": cat_conf,
            "link_override": {
                'filter': 'PPSFilter', 'sort': 'jd', "select": "first"
                }
        },
        {
            "unit": "T2LSPhotoZTap",
            "config": dl_config,
            "link_override": {
                'filter': 'PPSFilter', 'sort': 'jd', "select": "first"
                }
        },

    ]
}

In [ ]:
# Need at least a few of those, for bts z, for ampel z low for ampel z high
# This would be for any redshift
sncosmo_conf = {
    "sncosmo_model_name" : "salt2",
    "redshift_kind" : 'T2DigestRedshifts',
    "max_ampelz_group" : 7,      # For this purpose we use any available redshift
    "apply_mwcorrection": True,
    "plot_props": None,
    "t2_dependency": [
        {
            "unit": "T2DigestRedshifts",
            "config": ampelz_conf,
        },
    ], 
}
sncosmo_btsconf = {
    "sncosmo_model_name" : "salt2",
    "redshift_kind" : 'T2MatchBTS',
    "apply_mwcorrection": True,
    "plot_props": None,
    "t2_dependency": [
        {
            "unit": "T2MatchBTS",
        },
    ], 
}

In [ ]:
parsnip_config = {
    'parsnip_model':'/home/jnordin/data/parsnip/parsnip_ps1_ztf.pt', 
    'parsnip_classifier':'/home/jnordin/data/parsnip/ztf_classifier.pkl', 
    "redshift_kind" : 'T2DigestRedshifts',
    "max_ampelz_group" : 7,      # For this purpose we use any available redshift
    "t2_dependency": [
        {
            "unit": "T2DigestRedshifts",
            "config": ampelz_conf,
        },
    ], 
}

In [ ]:
directives = [
        {
            "channel": channel,
            "filter": {
                "unit": "SimpleDecentFilter",
                "config": filter_config,
                "on_stock_match": 'bypass',
            },
            "ingest": {
                "mux": {
                    "unit": "ZiArchiveMuxer",
                    "config": {
                        "history_days": 999,
                        "future_days": 999
                    },
                    "combine": [
                        {
                            "unit": "ZiT1Combiner",
                            "state_t2": [
                                {
                                    "unit": "T2DigestRedshifts",
                                    "config": ampelz_conf,
                                },
                                {
                                    "unit": "T2MatchBTS",
                                },
                                {
                                    "unit": "T2RunSncosmo",
                                    "config": sncosmo_conf,
                                },                            
#                                {
#                                    "unit": "T2RunSncosmo",
#                                    "config": sncosmo_zconf,
#                                },                            
                                {
                                    "unit": "T2RunSncosmo",
                                    "config": sncosmo_btsconf,
                                },                            
                                {
                                    "unit": "T2RunParsnip",
                                    "config": parsnip_config,
                                },                            
                            ]
                        }
                    ],
                    "insert": {
                        "point_t2": [
                            {
                                'unit': 'T2CatalogMatch',
                                'config': cat_conf,
                                "ingest": {
                                    'filter': 'PPSFilter', 'sort': 'jd', 'select': 'first'
                                }
                            },
                            {
                                'unit': 'T2LSPhotoZTap',
                                'config': dl_config,
                                "ingest": {
                                    'filter': 'PPSFilter', 'sort': 'jd', 'select': 'first'
                                }
                            },
                        ],
                    }

                }
            }
        }
    ]

In [ ]:
k = 0
while(k<30):
    print(k)

    try:
        ac = ctx.new_context_unit(
            unit = "AlertConsumer",
            process_name = "AP_test",
            iter_max = ITER_MAX,
            log_profile = os.environ.get('log_profile', 'debug'),
            shaper = "ZiDataPointShaper",
            compiler_opts = "ZiCompilerOptions",
            supplier = {
                "unit": "ZiAlertSupplier",
                'config': {
                    'deserialize': None,
                    'loader': {
                        'unit': 'ZTFArchiveAlertLoader',
                        'config': loader_config
                    }
                }
            },
            directives = directives
        )
        n = ac.run()
        print(f"{n} alerts processed for channel {channel}")
        
    except HTTPError as e:
        print(e)
        status_code = e.response.status_code
        print(status_code)
        print('trying to sleep some')
        time.sleep(240)
    except ConnectionError as e:
        print(e)
        print('prob just some temporary connection loss, try again')
#        time.sleep(120)

    
    k += 1

Why could we miss some BTS objects? If they are too bright, if they have too few detections, if they are far from PS objects or if they are close to the galactic plane (slightly more conservative cut for efficiency).